# 고객이탈 예측분석

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

# 1단계 데이터준비하기

In [2]:
# Read the data set into a pandas DataFrame
churn = pd.read_csv('churn.csv', sep=',', header=0)
churn.head()

State  Account Length  Area Code     Phone Int'l Plan VMail Plan  \
0    KS             128        415  382-4657         no        yes   
1    OH             107        415  371-7191         no        yes   
2    NJ             137        415  358-1921         no         no   
3    OH              84        408  375-9999        yes         no   
4    OK              75        415  330-6626        yes         no   

   VMail Message  Day Mins  Day Calls  Day Charge  ...  Eve Calls  Eve Charge  \
0             25     265.1        110       45.07  ...         99       16.78   
1             26     161.6        123       27.47  ...        103       16.62   
2              0     243.4        114       41.38  ...        110       10.30   
3              0     299.4         71       50.90  ...         88        5.26   
4              0     166.7        113       28.34  ...        122       12.61   

   Night Mins  Night Calls  Night Charge  Intl Mins  Intl Calls  Intl Charge  \
0       244.7           91         11.01       10.0           3         2.70   
1       254.4          103         11.45       13.7           3         3.70   
2       162.6          104          7.32       12.2           5         3.29   
3       196.9           89          8.86        6.6           7         1.78   
4       186.9          121          8.41       10.1           3         2.73   

   CustServ Calls  Churn?  
0               1  False.  
1               1  False.  
2               0  False.  
3               2  False.  
4               3  False.  

[5 rows x 21 columns]

In [3]:
#lower함수와 리스트 축약을 이용하여 모든 열 헤더를 소문자로 변환
churn.columns = [heading.lower() for heading in \
churn.columns.str.replace(' ', '_').str.replace("\'", "").str.strip('?')]
#헤더에 있는 공백을 밑줄로 대체하고, 헤더에 있는 작은 따옴표를 없앤다
#chun?과 같이 물음표가 있는 열 헤더에서 물음표 제거
churn.head()

state  account_length  area_code     phone intl_plan vmail_plan  \
0    KS             128        415  382-4657        no        yes   
1    OH             107        415  371-7191        no        yes   
2    NJ             137        415  358-1921        no         no   
3    OH              84        408  375-9999       yes         no   
4    OK              75        415  330-6626       yes         no   

   vmail_message  day_mins  day_calls  day_charge  ...  eve_calls  eve_charge  \
0             25     265.1        110       45.07  ...         99       16.78   
1             26     161.6        123       27.47  ...        103       16.62   
2              0     243.4        114       41.38  ...        110       10.30   
3              0     299.4         71       50.90  ...         88        5.26   
4              0     166.7        113       28.34  ...        122       12.61   

   night_mins  night_calls  night_charge  intl_mins  intl_calls  intl_charge  \
0       244.7           91         11.01       10.0           3         2.70   
1       254.4          103         11.45       13.7           3         3.70   
2       162.6          104          7.32       12.2           5         3.29   
3       196.9           89          8.86        6.6           7         1.78   
4       186.9          121          8.41       10.1           3         2.73   

   custserv_calls   churn  
0               1  False.  
1               1  False.  
2               0  False.  
3               2  False.  
4               3  False.  

[5 rows x 21 columns]

In [4]:
#이탈여부를 이진으로 나타내는 churn01이라는 새로운 열 생성
churn['churn01'] = np.where(churn['churn'] == 'True.', 1., 0.)#churn 열을 기반으로 True이면 1, False면 0이 할당
print(churn.head())
print(churn.describe())

  state  account_length  area_code     phone intl_plan vmail_plan  \
0    KS             128        415  382-4657        no        yes   
1    OH             107        415  371-7191        no        yes   
2    NJ             137        415  358-1921        no         no   
3    OH              84        408  375-9999       yes         no   
4    OK              75        415  330-6626       yes         no   

   vmail_message  day_mins  day_calls  day_charge  ...  eve_charge  \
0             25     265.1        110       45.07  ...       16.78   
1             26     161.6        123       27.47  ...       16.62   
2              0     243.4        114       41.38  ...       10.30   
3              0     299.4         71       50.90  ...        5.26   
4              0     166.7        113       28.34  ...       12.61   

   night_mins  night_calls  night_charge  intl_mins  intl_calls  intl_charge  \
0       244.7           91         11.01       10.0           3         2.70   
1   

# 2단계 탐색적 분석

#### 두 그룹간 차이점

- 전체데이터를 유지고객과 이탈고객으로 그룹화해서 기술통계를 구해보자
- 그룹별로 6개열에 대해 세가지 통계량(개수, 평균, 표준편차)를 산출한다

In [5]:

print(churn.groupby(['churn'])[['day_charge', 'eve_charge', 'night_charge', 
                                'intl_charge', 'account_length', 'custserv_calls']].agg(['count', 'mean', 'std']))

       day_charge                       eve_charge                       \
            count       mean        std      count       mean       std   
churn                                                                     
False.       2850  29.780421   8.530835       2850  16.918909  4.274863   
True.         483  35.175921  11.729710        483  18.054969  4.396762   

       night_charge                     intl_charge                      \
              count      mean       std       count      mean       std   
churn                                                                     
False.         2850  9.006074  2.299768        2850  2.743404  0.751784   
True.           483  9.235528  2.121081         483  2.889545  0.754152   

       account_length                       custserv_calls                      
                count        mean       std          count      mean       std  
churn                                                                           
False

#### 변수별로 서로 다른 통계량구하기
- 처음 4개 변수에 대해서는 평균과 표준편차를 다음  2개 변수에 대해서는 개수 최소값 최대값을 구한다
- churn열을 기준으로 그룹화하므로 그룹별로 통계량이 산출됨

In [6]:
print(churn.groupby(['churn']).agg({'day_charge' : ['mean', 'std'], 
				'eve_charge' : ['mean', 'std'],
				'night_charge' : ['mean', 'std'],
				'intl_charge' : ['mean', 'std'],
				'account_length' : ['count', 'min', 'max'],
				'custserv_calls' : ['count', 'min', 'max']}))

       day_charge            eve_charge           night_charge            \
             mean        std       mean       std         mean       std   
churn                                                                      
False.  29.780421   8.530835  16.918909  4.274863     9.006074  2.299768   
True.   35.175921  11.729710  18.054969  4.396762     9.235528  2.121081   

       intl_charge           account_length          custserv_calls          
              mean       std          count min  max          count min max  
churn                                                                        
False.    2.743404  0.751784           2850   1  243           2850   0   8  
True.     2.889545  0.754152            483   1  225            483   0   9  


- 전체요금을 뜻하는 새로운 변수 : total_charges를 기준으로, 데이터를 같은 폭의 5개 구간으로 그룹화한다음
- custser_calls변수(고객서비스센터 전화 횟수)의 데이터에 대해 5가지 통계량을 구해보자

In [27]:
# 전체요금을 기준으로 데이터를 같은 폭의 5개 구간으로 그룹화한 다음 "고객서비스센터 전화횟수"의 데이터에 대해 5가지 통계량을 구해보자
churn['total_charges'] = churn['day_charge'] + churn['eve_charge'] + \
						 churn['night_charge'] + churn['intl_charge']

factor_cut = pd.cut(churn.total_charges, 5, precision=2) #cut함수를 이용하여 total_charges변수의 데이터를 5개 구간으로 나누기

def get_stats(group):#그룹의 통계량을 반환하는 함수를 정의하자
	return {'min' : group.min(), 'max' : group.max(),
			'count' : group.count(), 'mean' : group.mean(),
			'std' : group.std()}
grouped = churn.custserv_calls.groupby(factor_cut) # custserv_call변수의 데이터를 위에서 구한 5개 구간으로 나눈다. 
print(grouped.apply(get_stats).unstack()) #get_stats()함수를 적용하여 이 5개 그룹에 대한 통계량을 산출한다. 

                min  max   count      mean       std
total_charges                                       
(22.86, 37.57]  0.0  5.0    70.0  1.528571  1.348337
(37.57, 52.22]  0.0  7.0   742.0  1.564690  1.305234
(52.22, 66.86]  0.0  9.0  1726.0  1.581692  1.326646
(66.86, 81.51]  0.0  9.0   735.0  1.523810  1.295209
(81.51, 96.15]  0.0  5.0    60.0  1.516667  1.359108


- custserv_calls변수의 데이터에 대해 5가지 통계량을 구하는 다른 방식
- 이번에는 qcut함수를 사용하여 account_length열의 사분위수를 기준으로 같은 폭이 아니라 같은 크기로 구간을 나누어 보자!

In [28]:
# account_length 열의 사분위수를 기준으로 분할한 뒤,  
# 그룹통계량 구하기!
factor_qcut = pd.qcut(churn.account_length, [0., 0.25, 0.5, 0.75, 1.])
grouped = churn.custserv_calls.groupby(factor_qcut)
print(grouped.apply(get_stats).unstack())

                min  max  count      mean       std
account_length                                     
(0.999, 74.0]   0.0  9.0  857.0  1.506418  1.251268
(74.0, 101.0]   0.0  7.0  847.0  1.604486  1.359888
(101.0, 127.0]  0.0  8.0  803.0  1.652553  1.358479
(127.0, 243.0]  0.0  9.0  826.0  1.491525  1.286970


- => 각 그룹에 거의 비슷한 개수의 관측값이 포함된다
- => qcut함수는 정수 또는 사분위수를 담은 배열을 인수로 취하는걸 볼 수 있다. 
- 정수10을 사용하여 십분위수를 지정할 수도 있다. 

#### 이진형 지시변수(indicator variables) 만들기

In [29]:
# intl_plan과 vmail_plan에 대한 이진형 지시변수를 만들고, 
# Churn 열과 병합하여 새로운 데이터 프레임 생성하기
intl_dummies = pd.get_dummies(churn['intl_plan'], prefix='intl_plan')
vmail_dummies = pd.get_dummies(churn['vmail_plan'], prefix='vmail_plan')
churn_with_dummies = churn[['churn']].join([intl_dummies, vmail_dummies])
print(churn_with_dummies.head())


    churn  intl_plan_no  intl_plan_yes  vmail_plan_no  vmail_plan_yes
0  False.             1              0              0               1
1  False.             1              0              0               1
2  False.             1              0              1               0
3  False.             0              1              1               0
4  False.             0              1              1               0


- 열을 사분위수를 기준으로 분할한뒤, 각 사분위수에 대한 이진형 지시변수를 만들자

In [30]:
#qcut함수로 total_charges열을 사분위수를 기준으로 분할하고 분할된 각 사분위수에 이름을 붙인다. 
qcut_names = ['1st_quartile', '2nd_quartile', '3rd_quartile', '4th_quartile']
total_charges_quartiles = pd.qcut(churn.total_charges, 4, labels=qcut_names)

In [31]:
#get_dummies함수로 사분위수에 대한 이진형 지시변수를 만들고, 새로운 열 이름에 total_charges를 접두사로 붙였다
dummies = pd.get_dummies(total_charges_quartiles, prefix='total_charges')
churn_with_dummies = churn.join(dummies)#join함수를 사용하여 이 4개의 변수를 churn데이터프레임에 추가한다.
print(churn_with_dummies.head())

  state  account_length  area_code     phone intl_plan vmail_plan  \
0    KS             128        415  382-4657        no        yes   
1    OH             107        415  371-7191        no        yes   
2    NJ             137        415  358-1921        no         no   
3    OH              84        408  375-9999       yes         no   
4    OK              75        415  330-6626       yes         no   

   vmail_message  day_mins  day_calls  day_charge  ...  intl_calls  \
0             25     265.1        110       45.07  ...           3   
1             26     161.6        123       27.47  ...           3   
2              0     243.4        114       41.38  ...           5   
3              0     299.4         71       50.90  ...           7   
4              0     166.7        113       28.34  ...           3   

   intl_charge  custserv_calls   churn  churn01  total_charges  \
0         2.70               1  False.      0.0          75.56   
1         3.70               1  

## 피봇테이블 생성하기

In [32]:
# churn과 custserv_calls열을 기준으로 pivoting 그룹화하여 total_charges의 평균을 산출
print(churn.pivot_table(['total_charges'], index=['churn', 'custserv_calls']))
# 그 결과는 각 churn여부 및 각 custser_calls열에 따른 total_charges의 평균으로 이루어진 하나의 열이된다

                       total_charges
churn  custserv_calls               
False. 0                   58.429752
       1                   58.164391
       2                   57.534241
       3                   58.797195
       4                   64.318000
       5                   66.303077
       6                   62.150000
       7                   64.677500
       8                   64.670000
True.  0                   69.601087
       1                   70.723443
       2                   69.399080
       3                   68.931136
       4                   55.374474
       5                   52.845500
       6                   49.714286
       7                   50.578000
       8                   52.730000
       9                   70.390000


- #churn을 기준으로 행을 정의하고, custserv_calls을 기준으로 열을 정의하여 데이터를 재구조화 해보자

In [33]:
print(churn.pivot_table(['total_charges'], index=['churn'], columns=['custserv_calls']))

               total_charges                                              \
custserv_calls             0          1          2          3          4   
churn                                                                      
False.             58.429752  58.164391  57.534241  58.797195  64.318000   
True.              69.601087  70.723443  69.399080  68.931136  55.374474   

                                                             
custserv_calls          5          6        7      8      9  
churn                                                        
False.          66.303077  62.150000  64.6775  64.67    NaN  
True.           52.845500  49.714286  50.5780  52.73  70.39  


- 이번에는 custser_calls을 행, churn을 열의 기준으로 사용하고, 산출할 통계량과 결측 값을 대체할 값 부분합 표시여부를 지정한다

In [34]:
print(churn.pivot_table(['total_charges'], index=['custserv_calls'], columns=['churn'], \
						aggfunc='mean', fill_value='NaN', margins=True))

               total_charges                  
churn                 False.    True.      All
custserv_calls                                
0                    58.4298  69.6011  59.9043
1                    58.1644  70.7234  59.4618
2                    57.5342  69.3991  58.8942
3                    58.7972  68.9311  59.8366
4                     64.318  55.3745  60.2234
5                    66.3031  52.8455   58.147
6                      62.15  49.7143  54.2364
7                    64.6775   50.578  56.8444
8                      64.67    52.73     58.7
9                        NaN    70.39    70.39
All                  58.4488   65.356  59.4498


# 3단계 예측모형 만들기

### 로지스틱 회귀분석

- 고객 이탈 데이터셋의 종속변수는 이진형으로서 고객의 이탈 여부를 나타낸다
- 이처럼 종속변수의 값이 0과 1인 경우에는 선형회귀분석이 적합하지 않다

- 로지스틱회귀분석 : 이진형 종속변수와 독립변수 간의 관계를 로지스틱 함수를 이용하여 추정함
- 이 함수는 연속형인 값을 입력받아 0 과 1사이의 값으로 반환한다. 

- 로지스틱 회귀분석은 최대우도법(Maximum Likelihood Method)를 이용하여 계수를 추정한다
- 선형회귀분석 구문과 달리 로지스틱 회귀분석에서는 회귀식 대신 독립변수와 종속변수를 따로 할당한다. 

In [35]:
dependent_variable = churn['churn01']
independent_variables = churn[['account_length', 'custserv_calls', 'total_charges']]
#add_constant함수를 사용하여 입력변수에 전체값이 1인 열을 추가한다. 
independent_variables_with_constant = sm.add_constant(independent_variables, prepend=True)
independent_variables_with_constant.head()

const  account_length  custserv_calls  total_charges
0    1.0             128               1          75.56
1    1.0             107               1          59.24
2    1.0             137               0          62.29
3    1.0              84               2          66.80
4    1.0              75               3          52.09

# 4단계 학습하기

- 로지스틱 회귀모형에 학습시키고 그 결과를 logit_model에 할당한다. 

In [36]:
logit_model = sm.Logit(dependent_variable, independent_variables_with_constant).fit()
#logit_model = smf.glm(output_variable, input_variables, family=sm.families.Binomial()).fit()

Optimization terminated successfully.
         Current function value: 0.363480
         Iterations 7


In [37]:
#회귀계수, 표준오차 및 신뢰구간, 결정계수등을 출력
print(logit_model.summary())

                           Logit Regression Results                           
Dep. Variable:                churn01   No. Observations:                 3333
Model:                          Logit   Df Residuals:                     3329
Method:                           MLE   Df Model:                            3
Date:                Fri, 02 Oct 2020   Pseudo R-squ.:                  0.1216
Time:                        23:54:24   Log-Likelihood:                -1211.5
converged:                       True   LL-Null:                       -1379.1
Covariance Type:            nonrobust   LLR p-value:                 2.234e-72
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
const             -7.2205      0.394    -18.309      0.000      -7.993      -6.448
account_length     0.0012      0.001      0.927      0.354      -0.001       0.004
custserv_calls     0.4443      0.037

In [38]:
print("\nQuantities you can extract from the result:\n%s" % dir(logit_model)) # 모든 통계량을 리스트형태로 출력
print("\nCoefficients:\n%s" % logit_model.params) #회귀계수만
print("\nCoefficient Std Errors:\n%s" % logit_model.bse) #표준오차만


Quantities you can extract from the result:
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_cache', '_data_attr', '_get_endog_name', '_get_robustcov_results', '_use_t', 'aic', 'bic', 'bse', 'conf_int', 'cov_kwds', 'cov_params', 'cov_type', 'df_model', 'df_resid', 'f_test', 'fittedvalues', 'get_margeff', 'initialize', 'k_constant', 'llf', 'llnull', 'llr', 'llr_pvalue', 'load', 'mle_retvals', 'mle_settings', 'model', 'nobs', 'normalized_cov_params', 'params', 'pred_table', 'predict', 'prsquared', 'pvalues', 'remove_data', 'resid_dev', 'resid_generalized', 'resid_pearson', 'resid_response', 'save', 'scale', 'set_null_options', 'summary', 'summary2', 't_test', 't_test_pairwise', 

### 예측모형 해석

In [39]:
print("\ninvlogit(-7.2205 + 0.0012*mean(account_length) + 0.4443*mean(custserv_calls) + 0.0729*mean(total_charges))")


invlogit(-7.2205 + 0.0012*mean(account_length) + 0.4443*mean(custserv_calls) + 0.0729*mean(total_charges))


- 로지스틱 회귀분석에서 회귀계수의 해석은 선형회귀와 달리 직관적이지 않음!
- 로지스틱회귀는 S자 곡선이므로 독립변수가 한단위만큼 변할때 기대되는 종속변수의 변화가 일정하지 않기 때문

- 로지스틱 회귀모형에서 회귀계수가 종속변수의 성공 확률에 미치는 영향을 파악하기 위해서는?
- => 모형평가의 기준, 즉 오즈(odds)가 필요함. 오즈는 모든 독립변수가 0인경우에 성공확률에 미치는 영향을 나타냄
- 모든 독립변수를 평균으로 설정하여 모형을 평가해야함. 

- 선형회귀모형의 연속형 예측값을 0과 1의 확률로 변환해보자!

In [40]:
def inverse_logit(model_formula):
	from math import exp
	return (1.0 / (1.0 + exp(-model_formula)))*100.0

- 모든 독립변수를 평균으로 설정했을때의 예측값을 추정

In [41]:
at_means = float(logit_model.params[0]) + \
	float(logit_model.params[1])*float(churn['account_length'].mean()) + \
	float(logit_model.params[2])*float(churn['custserv_calls'].mean()) + \
	float(logit_model.params[3])*float(churn['total_charges'].mean())

In [42]:
print(churn['account_length'].mean())
print(churn['custserv_calls'].mean())
print(churn['total_charges'].mean())

101.06480648064806
1.5628562856285628
59.44975397539747


- at_means값을 로지스틱 함수에 입력하였을때 출력되는 결과를 출력

In [43]:
print(at_means)
print("Probability of churn when independent variables are at their mean values: %.2f" % inverse_logit(at_means))

-2.0679167809476997
Probability of churn when independent variables are at their mean values: 11.23


- -2.068의 로짓 역함수를 취하면 0.112이고, 따라서 이탈고객사이에서 세 독립변수의 평균이 똑같을 확률을 11.2%라고 말할 수 있다. 

- 마찬가지로, 독립변수 중 하나의 단위변화에 따른 종속변수의 변화를 평가하려면?
- 독립변수 중 하나를 평균에 가까운 값으로 변경하여 확률의 차이를 평가할 수 이싿. 
- 예를 들어 custserv_calls열을 기준으로 살펴보자

In [44]:
cust_serv_mean = float(logit_model.params[0]) + \
	float(logit_model.params[1])*float(churn['account_length'].mean()) + \
	float(logit_model.params[2])*float(churn['custserv_calls'].mean()) + \
	float(logit_model.params[3])*float(churn['total_charges'].mean())
		
cust_serv_mean_minus_one = float(logit_model.params[0]) + \
		float(logit_model.params[1])*float(churn['account_length'].mean()) + \
		float(logit_model.params[2])*float(churn['custserv_calls'].mean()-1.0) + \
		float(logit_model.params[3])*float(churn['total_charges'].mean())

print(cust_serv_mean)
print(churn['custserv_calls'].mean()-1.0)
print(cust_serv_mean_minus_one)

#둘 모두 로지스틱 함수를 취하여 cust_serv_mean의 추정값에서 cust_serv_means_minus_one의 추정값을 뺀다
print("Probability of churn when account length changes by 1: %.2f" % (inverse_logit(cust_serv_mean) - inverse_logit(cust_serv_mean_minus_one)))


-2.0679167809476997
0.5628562856285628
-2.512239499484104
Probability of churn when account length changes by 1: 3.72


#### 결과해석:
- cust_serv_mean의 값은 앞의 at_means의 값인 -2.068과 같이 나오고,  cust_serv_mean_minus_one은 -2.512가 나온다
- 두값에 각각 로지스틱함수를 취하고 뺀 결과는 0.037이므로,
- custserv_calls의 평균 근처에서 고객 서비스호출이 한 단위 증가하는 경우, 이탈확률이 3.72%증가한다고 말 할 수 있다. 

# 5단계: 예측하기

- 로지스틱회귀모형을 사용하여 새로운 관측값에 대해 예측해보자

In [45]:
#기존 데이터셋의 첫 10개를 가지고, 새로운 관측값 데이터셋을 만들자
new_observations = churn.ix[churn.index.isin(range(10)), independent_variables.columns]
new_observations_with_constant = sm.add_constant(new_observations, prepend=True)
y_predicted = logit_model.predict(new_observations_with_constant)
y_predicted_rounded = [round(score, 2) for score in y_predicted]
print(y_predicted_rounded)

AttributeError: 'DataFrame' object has no attribute 'ix'